In [1]:

import torch
from torch import nn

from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader
from custom_image_folder import CustomImageFolder
import chromadb
from custom_cnn.networks.model_skipped_18_layer_for_embedding import Skipped18LayerForEmbbeding
from custom_cnn import utility_functions

# Custom ResNet18 (Skipped18LayerForEmbedding)
In this script a trained instance of our Skipped18LayerForEmbedding is used to obtain embbedings for all Cifar10 samples.
With the embeddings of the training samples the test samples are classified via KNN.
The accuracy of that KNN classifier on the test set is recorded.
## Prepare embedding model

In [2]:
model = Skipped18LayerForEmbbeding()
model.network.load_state_dict(
    torch.load("/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn/savedmodels/Skipped18LayerForEmbbeding_20240114_2315/model_state_dict", map_location=torch.device(utility_functions.get_default_device()))
)

Set up Modular Skip
in: 16, out: 16, stride: (1, 1)
No skip conv needed, use identity
Set up Modular Skip
in: 16, out: 32, stride: (2, 2)
Create skip conv
Set up Modular Skip
in: 32, out: 32, stride: (1, 1)
No skip conv needed, use identity
Set up Modular Skip
in: 32, out: 64, stride: (2, 2)
Create skip conv
Set up Modular Skip
in: 64, out: 64, stride: (1, 1)
No skip conv needed, use identity
Set up Modular Skip
in: 64, out: 128, stride: (2, 2)
Create skip conv
Set up Modular Skip
in: 128, out: 128, stride: (1, 1)
No skip conv needed, use identity
Set up Modular Skip
in: 128, out: 128, stride: (1, 1)
No skip conv needed, use identity


<All keys matched successfully>

In [3]:
print(F"type of model {type(model)}")
print(f"{model.fc}")

type of model <class 'custom_cnn.networks.model_skipped_18_layer_for_embedding.Skipped18LayerForEmbbeding'>
Sequential(
  (0): Linear(in_features=128, out_features=10, bias=True)
  (1): Softmax(dim=1)
)


In [4]:
# Replace last FC layer by identity mapping so that the model outputs vectors of dim 512
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

model.fc = Identity()

## Prepare vector store

In [7]:
chroma_client = chromadb.PersistentClient(path="/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/embeddings/embedding_data/embedding_store")

collection = chroma_client.create_collection(name="custom_resnet18_embeddings_2div_38ce")
#collection = chroma_client.get_collection(name="my_collection")
#print(f"Samples in db: {len(collection.get()['embeddings'])}")

UniqueConstraintError: Collection custom_resnet18_embeddings_2div_38ce already exists

In [8]:
def get_embeddings_and_store(embedding_model, data_loader, chroma_collection, dataset_type):
    count = 0
    for batch in data_loader:
        print(count)
        batch_paths_and_images, batch_labels = batch
        batch_labels = batch_labels.tolist()
        batch_paths = list(batch_paths_and_images[0])
        batch_images = batch_paths_and_images[1]
        
        out = embedding_model(batch_images)
        
        chroma_collection.add(
            embeddings = out.tolist(),
            metadatas = [{'path':x[0], 'label':x[1], 'dataset_type':dataset_type} for x in zip(batch_paths, batch_labels)],
            ids= [dataset_type + str(x) for x in list(range( count * 256, count * 256 + len(batch_labels)))]
        )

        count += 1


## Get embeddings of train dataqset

In [9]:
# Use custom Dataset that returns image path in get item
train_dataset = CustomImageFolder(
    root="/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn/data/cifar10/train",
    transform=ToTensor()
)
# Create DataLoader
train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=False  # should be True if cuda is available
)
# feed through embedding model
model.eval()
get_embeddings_and_store(model, train_data_loader, collection, dataset_type="train")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195


In [10]:
print(f"Samples in db: {len(collection.get()['ids'])}")
print(collection.get()['ids'][0])
print(collection.get()['ids'][-1])

Samples in db: 50000
train0
train49999


## Get embeddings of test dataset

In [11]:
# Use custom Dataset that returns image path in get item
test_dataset = CustomImageFolder(
    root="/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn/data/cifar10/test",
    transform=ToTensor()
)
# Create DataLoader
test_data_loader = DataLoader(
    dataset=test_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4,
    pin_memory=False  # should be True if cuda is available
)
# feed through embedding model
get_embeddings_and_store(model, test_data_loader, chroma_collection=collection, dataset_type="test")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [12]:
print(f"Samples in db: {len(collection.get()['ids'])}")
print(collection.get()['ids'][50000])
print(collection.get()['ids'][-1])

Samples in db: 60000
test0
test9999


In [13]:
collection.get(include=['embeddings', 'documents', 'metadatas'])['embeddings'][0]

[13.240062713623047,
 11.977143287658691,
 12.995529174804688,
 13.166513442993164,
 10.426230430603027,
 11.87973403930664,
 13.181221008300781,
 10.080196380615234,
 13.002662658691406,
 15.055418014526367,
 4.618042945861816,
 4.911233901977539,
 4.804396629333496,
 5.186165809631348,
 5.3276472091674805,
 4.978082656860352,
 4.829348564147949,
 4.917346477508545,
 4.645643711090088,
 4.926084995269775,
 5.579319953918457,
 4.879996299743652,
 4.494848251342773,
 4.752858638763428,
 4.558714389801025,
 4.934080600738525,
 4.900467395782471,
 5.547007083892822,
 4.390097618103027,
 4.92465353012085,
 5.082063674926758,
 5.029162883758545,
 4.863217830657959,
 4.898233890533447,
 5.611480236053467,
 5.420427322387695,
 5.032962799072266,
 5.295525550842285,
 5.658536434173584,
 4.490195274353027,
 4.854650974273682,
 5.0996551513671875,
 4.923774242401123,
 5.083693504333496,
 4.978439807891846,
 5.274734020233154,
 4.535444736480713,
 5.094045639038086,
 5.032757759094238,
 4.8885574